In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os



import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn.metrics import mean_squared_error


# This one is only available in the development verison of statsmodels
# Run:
#    pip install git+https://github.com/statsmodels/statsmodels.git
# to install the development version.
from statsmodels.tsa.statespace.sarimax import SARIMAX


# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

from scipy import signal
from scipy import stats

sys.path.append("..")
from eda_utilities import print_eda_stats
from data_pipeline import dockless_data_pipeline
from data_pipeline import weather_data_pipeline

In [2]:
# Read the dockless data from S3 bucket
# Load the data and and call the clean method automatically to call the work flow
dockless_data = pd.read_csv("https://s3.amazonaws.com/sameera-bucket-1/dockless_mobility/raw_data/Austin_Dockless_Vehicle_Trips.csv")
dockless_vehicles_columns = dockless_data.columns
print_eda_stats(dockless_data, dockless_vehicles_columns, "Austin Dockless Mobility Data")
# Call the dockless_data_pipeline()
dockless_data_pipeline(dockless_data)
dockless_vehicles_columns = dockless_data.columns
print_eda_stats(dockless_data, dockless_vehicles_columns, "Austin Dockless Mobility Data")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


+--------------------------+---------+-------------+---------------+--------------+-------------------+
|       Column Name        |  dtype  | Total_Count | Unique_Values | df_totalrows | Null Value counts |
+--------------------------+---------+-------------+---------------+--------------+-------------------+
|            ID            |  object |   2746505   |    2746505    |   2746505    |         0         |
|        Device ID         |  object |   2746505   |     26999     |   2746505    |         0         |
|       Vehicle Type       |  object |   2746504   |       3       |   2746505    |         1         |
|      Trip Duration       | float64 |   2746504   |     10521     |   2746505    |         1         |
|      Trip Distance       | float64 |   2746504   |     16125     |   2746505    |         1         |
|        Start Time        |  object |   2746504   |     25694     |   2746505    |         1         |
|         End Time         |  object |   2746504   |     25768  

In [3]:
#Read the weather data from S3 bucket
weather_data = pd.read_csv("https://s3.amazonaws.com/sameera-bucket-1/dockless_mobility/raw_data/Austin_Bergstom_Airport_Weather.csv")
weather_columns = weather_data.columns
print_eda_stats(weather_data, weather_columns, "Austin Weather Data")
weather_data_pipeline(weather_data)
weather_columns = weather_data.columns
print_eda_stats(weather_data, weather_columns, "Austin Weather Data")

+-----------------+---------+-------------+---------------+--------------+-------------------+
|   Column Name   |  dtype  | Total_Count | Unique_Values | df_totalrows | Null Value counts |
+-----------------+---------+-------------+---------------+--------------+-------------------+
|     STATION     |  object |     436     |       1       |     436      |         0         |
|       NAME      |  object |     436     |       1       |     436      |         0         |
|     LATITUDE    | float64 |     436     |       1       |     436      |         0         |
|    LONGITUDE    | float64 |     436     |       1       |     436      |         0         |
|    ELEVATION    | float64 |     436     |       1       |     436      |         0         |
|       DATE      |  object |     436     |      436      |     436      |         0         |
|       AWND      | float64 |     435     |       73      |     436      |         1         |
| AWND_ATTRIBUTES |  object |     435     |       

In [4]:
# Drop all records where  trip distance is less .1 miles and greater than 500 miles (1609.34 mts - 804673 mts)
dockless_data = dockless_data[(dockless_data['Trip Distance'] > 1609.34) & (dockless_data['Trip Distance'] < 804673)]
dockless_data = dockless_data[(dockless_data['Trip Duration'] > 60) & (dockless_data['Trip Duration'] < 86400)]

In [5]:
# Add to pipeline so it easy for calculations (Done)
# dockless_data['count'] = 1

# Have only weather data which is from the start date of the scooter data
trip_counts = dockless_data.groupby('START_DATE').count()['count']
weather_data= weather_data[weather_data['DATE'] >= trip_counts.index.min()]
weather_data= weather_data[weather_data['DATE'] <= trip_counts.index.max()]
weather_data = weather_data.reset_index()

In [6]:
grouped_origin_cell_id = dockless_data.groupby(['Origin Cell ID']).agg('count')
grouped_origin_cell_id = grouped_origin_cell_id.sort_values(['Trip Duration'], ascending=False)
top_origin_cell_ids = grouped_origin_cell_id['Trip Duration']
# Rename the column Trip Duration to Trip Counts
top_origin_cell_ids = top_origin_cell_ids.to_frame().reset_index()
top_50_origin_cells = top_origin_cell_ids.head(50)
#top_50_origin_cells.to_csv("../../output/EDA/Top_50_Origin_Cells_Total_Rides.csv")
top_10_origin_cells = top_50_origin_cells[0:10]
top_10_origin_cells = top_10_origin_cells['Origin Cell ID'].values
top_10_origin_cells_list = top_10_origin_cells.tolist()

In [7]:
# Make the DateTime as index
dockless_data = dockless_data.reset_index()
dockless_data = dockless_data.set_index("Start Time")
origin_cell_data = dockless_data[['Origin Cell ID', 'count']]

In [8]:
top_10_origin_cell_data = origin_cell_data[origin_cell_data['Origin Cell ID'].isin(top_10_origin_cells_list)]

In [9]:
# Top 3 origin cell data
cell_014706_data = top_10_origin_cell_data[top_10_origin_cell_data['Origin Cell ID'] == '014706']
cell_013650_data = top_10_origin_cell_data[top_10_origin_cell_data['Origin Cell ID'] == '013650']
cell_013176_data = top_10_origin_cell_data[top_10_origin_cell_data['Origin Cell ID'] == '013176']

In [10]:
trip_counts_cell_014706 = cell_014706_data.groupby([cell_014706_data.index.get_level_values(0),'Origin Cell ID']).count()
trip_counts_cell_014706 = trip_counts_cell_014706.unstack(level=1)
trip_counts_cell_014706 = trip_counts_cell_014706.fillna(0)

# Make a regular dataframe for processing the Time Series
t2 = trip_counts_cell_014706.reset_index()['count']
counts = t2['014706'].values
data = {'Start Time':trip_counts_cell_014706.index.values, '014706':counts} 
trip_counts_cell_014706_new = pd.DataFrame(data)
data_014706 = trip_counts_cell_014706_new.set_index("Start Time")



In [12]:
# data after '2018-07-15' as the data is not consistent before that

data_014706 = data_014706[(data_014706.index > '2018-07-15')]
data_014706.head(5)

,014706
Start Time,
2018-07-15 00:30:00,1
2018-07-15 07:45:00,1
2018-07-15 09:15:00,1
2018-07-15 09:30:00,2
2018-07-15 10:45:00,1


In [13]:
# Resample for Hourly and fill with sum of the rides
data_014706_hourly = data_014706.resample('H', how='sum')
data_014706_hourly = data_014706_hourly.fillna(0)
data_014706_hourly.head(20)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  


,014706
Start Time,
2018-07-15 00:00:00,1
2018-07-15 01:00:00,0
2018-07-15 02:00:00,0
2018-07-15 03:00:00,0
2018-07-15 04:00:00,0
2018-07-15 05:00:00,0
2018-07-15 06:00:00,0
2018-07-15 07:00:00,1
2018-07-15 08:00:00,0


In [20]:
data_014706_hourly.tail(20)

,014706
Start Time,
2019-02-07 22:00:00,0
2019-02-07 23:00:00,0
2019-02-08 00:00:00,1
2019-02-08 01:00:00,1
2019-02-08 02:00:00,0
2019-02-08 03:00:00,0
2019-02-08 04:00:00,0
2019-02-08 05:00:00,0
2019-02-08 06:00:00,0


In [27]:
# Add the following features to the data 
'''
Month, Day of the week(M,T, W, T, F, S, Sun), Weekday, Weekend, Week, Hour, Year, Max Temp, Precipitation, 
The day of the week with Monday=0, Sunday=6.
'''
data_014706_hourly['MONTH'] = pd.DatetimeIndex(data_014706_hourly.index).month
data_014706_hourly['YEAR'] = pd.DatetimeIndex(data_014706_hourly.index).year
data_014706_hourly['HOUR'] = pd.DatetimeIndex(data_014706_hourly.index).hour
data_014706_hourly['DAY'] = pd.DatetimeIndex(data_014706_hourly.index).day
data_014706_hourly['WEEK'] = pd.DatetimeIndex(data_014706_hourly.index).week
data_014706_hourly['DAY_OF_WEEK'] = pd.DatetimeIndex(data_014706_hourly.index).weekday
data_014706_hourly['WEEKEND'] = ((pd.DatetimeIndex(data_014706_hourly.index).weekday) // 5 == 1).astype(float)
data_014706_hourly['WEEKDAY'] = ((pd.DatetimeIndex(data_014706_hourly.index).weekday) // 5 == 0).astype(float)
data_014706_hourly['DATE'] = pd.DatetimeIndex(data_014706_hourly.index).date

In [28]:
data_014706_hourly.head(5)

,014706,MONTH,YEAR,HOUR,DAY,WEEK,DAY_OF_WEEK,WEEKEND,WEEKDAY,DATE
Start Time,,,,,,,,,,
2018-07-15 00:00:00,1,7,2018,0,15,28,6,1.0,0.0,2018-07-15
2018-07-15 01:00:00,0,7,2018,1,15,28,6,1.0,0.0,2018-07-15
2018-07-15 02:00:00,0,7,2018,2,15,28,6,1.0,0.0,2018-07-15
2018-07-15 03:00:00,0,7,2018,3,15,28,6,1.0,0.0,2018-07-15
2018-07-15 04:00:00,0,7,2018,4,15,28,6,1.0,0.0,2018-07-15


In [32]:
# Merge the weather data into the hourly dataframe so you can add all the features

merged_data_014706_hourly = pd.merge(left=data_014706_hourly,right=weather_data, how='left',left_on='DATE', right_on='DATE')

merged_data_014706_hourly.head(5)


,014706,MONTH,YEAR,HOUR,DAY,WEEK,DAY_OF_WEEK,WEEKEND,WEEKDAY,DATE,index,AVG_DAILY_WIND_SPEED,PRECIPITATION,SNOW,AVG_TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
0,1,7,2018,0,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
1,0,7,2018,1,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2,0,7,2018,2,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
3,0,7,2018,3,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
4,0,7,2018,4,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0


In [38]:
merged_data_014706_hourly = data_014706_hourly.reset_index().merge(weather_data, on='DATE', how="left").set_index('Start Time')


merged_data_014706_hourly.head(10)


,014706,MONTH,YEAR,HOUR,DAY,WEEK,DAY_OF_WEEK,WEEKEND,WEEKDAY,DATE,index,AVG_DAILY_WIND_SPEED,PRECIPITATION,SNOW,AVG_TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
Start Time,,,,,,,,,,,,,,,,,
2018-07-15 00:00:00,1,7,2018,0,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 01:00:00,0,7,2018,1,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 02:00:00,0,7,2018,2,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 03:00:00,0,7,2018,3,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 04:00:00,0,7,2018,4,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 05:00:00,0,7,2018,5,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 06:00:00,0,7,2018,6,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 07:00:00,1,7,2018,7,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 08:00:00,0,7,2018,8,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0


In [29]:
weather_data.head(5)

,index,DATE,AVG_DAILY_WIND_SPEED,PRECIPITATION,SNOW,AVG_TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
0,93,2018-04-04,12.53,0.00,0.0,62.0,71.0,49.0
1,94,2018-04-05,9.17,0.00,0.0,64.0,78.0,53.0
2,95,2018-04-06,12.75,0.30,0.0,75.0,86.0,64.0
3,96,2018-04-07,17.22,0.01,0.0,57.0,64.0,41.0
4,97,2018-04-08,6.49,0.00,0.0,47.0,65.0,41.0


In [39]:
# Create the test train split the same way as the univariate time series so this is the same as ARIMA series
size = int(len(merged_data_014706_hourly) * 0.66)
train, test = merged_data_014706_hourly[0:size], merged_data_014706_hourly[size:len(merged_data_014706_hourly)]

In [40]:
train.head(5)

,014706,MONTH,YEAR,HOUR,DAY,WEEK,DAY_OF_WEEK,WEEKEND,WEEKDAY,DATE,index,AVG_DAILY_WIND_SPEED,PRECIPITATION,SNOW,AVG_TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
Start Time,,,,,,,,,,,,,,,,,
2018-07-15 00:00:00,1,7,2018,0,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 01:00:00,0,7,2018,1,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 02:00:00,0,7,2018,2,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 03:00:00,0,7,2018,3,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0
2018-07-15 04:00:00,0,7,2018,4,15,28,6,1.0,0.0,2018-07-15,195,10.29,0.0,0.0,86.0,98.0,75.0


In [41]:
test.head(5)

,014706,MONTH,YEAR,HOUR,DAY,WEEK,DAY_OF_WEEK,WEEKEND,WEEKDAY,DATE,index,AVG_DAILY_WIND_SPEED,PRECIPITATION,SNOW,AVG_TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
Start Time,,,,,,,,,,,,,,,,,
2018-11-29 18:00:00,16,11,2018,18,29,48,3,0.0,1.0,2018-11-29,332,10.51,0.0,0.0,61.0,76.0,47.0
2018-11-29 19:00:00,9,11,2018,19,29,48,3,0.0,1.0,2018-11-29,332,10.51,0.0,0.0,61.0,76.0,47.0
2018-11-29 20:00:00,2,11,2018,20,29,48,3,0.0,1.0,2018-11-29,332,10.51,0.0,0.0,61.0,76.0,47.0
2018-11-29 21:00:00,0,11,2018,21,29,48,3,0.0,1.0,2018-11-29,332,10.51,0.0,0.0,61.0,76.0,47.0
2018-11-29 22:00:00,2,11,2018,22,29,48,3,0.0,1.0,2018-11-29,332,10.51,0.0,0.0,61.0,76.0,47.0


In [42]:
y_train = train.pop('014706').values
X_train = train.values
y_test = test.pop('014706').values
X_test = test.values

In [ ]:
rf = RandomForest(num_trees=10, num_features=(X.shape[1]//3))
rf.fit(X_train, y_train)
print("Random Forest score:", rf.score(X_test, y_test))